In [26]:
import requests
import base64

CLIENT_ID = 'd1483272e1a54c258b3433c1024df7a7'
CLIENT_SECRET = 'edc435f75e4a40ca8fde16edd568866f'

client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization':f'Basic {client_credentials_base64.decode()}'
}

data = {
    'grant_type':'client_credentials'
}

response = requests.post(token_url, data =data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Success")
    
else:
    print("error")
    


Success


In [27]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

def get_trending_playlist_data(playlist_id,access_token):
    # set up Spotipy with access token
    sp = spotipy.Spotify(auth=access_token)
    
    playlist_tracks = sp.playlist_tracks(playlist_id,fields = 'items(track(id, name, artists, album(id, name)))')
    
    music_data = []
    
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ','.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']
        
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None
        
        try:
            album_info = sp.album(album_id) if album_id != "Not available" else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None
            
        
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
            
        except:
            popularity = None
            
        track_data = {
            'track name':track_name,
            'artists':artists,
            'album name':album_name,
            'album id':album_id,
            'track id':track_id,
            'popularity':popularity,
            'release date':release_date,
            'duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'explicit': track_info.get('explicit',None),
            'external urls':track_info.get('external_urls',{}).get('spotify',None),
            'danceability':audio_features['danceability'] if audio_features else None,
            'energy':audio_features['energy'] if audio_features else None,
            'key':audio_features['key'] if audio_features else None,
            'loudness':audio_features['loudness'] if audio_features else None,
            'mode':audio_features['mode'] if audio_features else None,
            'speechiness':audio_features['speechiness'] if audio_features else None,
            'acousticness':audio_features['acousticness'] if audio_features else None,
            'instrumentalness':audio_features['instrumentalness'] if audio_features else None,
            'liveness':audio_features['liveness'] if audio_features else None,
            'valence':audio_features['valence'] if audio_features else None,
            'tempo': audio_features['tempo'] if audio_features else None
        }
        
        music_data.append(track_data)
        
    df = pd.DataFrame(music_data)
    
    return df

In [28]:
playlist_id = "5xfL1z44xoLhse7zV0icGK"

music_df = get_trending_playlist_data(playlist_id=playlist_id,access_token=access_token)

In [54]:
music_df

,track name,artists,album name,album id,track id,popularity,release date,duration (ms),explicit,external urls,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Corazón (feat. Nego do Borel),"Maluma,Nego do Borel",F.A.M.E.,6MoaDh76Fsg0ogW2l7HAFx,5cepAtqnEQ6yVG6088zMMu,72,2018-05-18,184720,False,https://open.spotify.com/track/5cepAtqnEQ6yVG6...,0.722,0.738,9,-6.073,0,0.2470,0.32800,0.000015,0.1980,0.748,198.075
1,El Préstamo,Maluma,F.A.M.E.,6MoaDh76Fsg0ogW2l7HAFx,2XVilv6s9GETnw9PG4T9Mz,64,2018-05-18,219600,False,https://open.spotify.com/track/2XVilv6s9GETnw9...,0.740,0.740,5,-3.647,0,0.0686,0.20500,0.000000,0.1070,0.717,93.950
2,El Perdedor,Maluma,"Pretty Boy, Dirty Boy",3PomvFR694SrCSZzDWMWV7,1iEwyiSLAunPR6uouANE0O,77,2015-10-30,206693,False,https://open.spotify.com/track/1iEwyiSLAunPR6u...,0.702,0.730,0,-5.173,0,0.0990,0.42500,0.000001,0.1480,0.879,173.967
3,Chantaje (feat. Maluma),"Shakira,Maluma",El Dorado,6bUxh58rYTL67FS8dyTKMN,6mICuAdrwEjh6Y6lroV2Kg,78,2017-05-26,195840,False,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,0.852,0.773,8,-2.921,0,0.0776,0.18700,0.000030,0.1590,0.907,102.034
4,Hawái,Maluma,PAPI JUANCHO,0p2yf6DucEgvj8Uk8KXJJv,1yoMvmasuxZfqHEipJhRbp,80,2020-08-21,199112,False,https://open.spotify.com/track/1yoMvmasuxZfqHE...,0.783,0.727,6,-3.454,0,0.3890,0.18500,0.000025,0.0626,0.558,180.067
5,Hawái - Remix,"Maluma,The Weeknd",Hawái (Remix),1Ag9EPbzibwzz0S0WpgX3v,0GzuHFG4Ql6DoyxFRnIk3F,69,2020-11-05,200482,False,https://open.spotify.com/track/0GzuHFG4Ql6Doyx...,0.766,0.788,6,-3.128,0,0.3120,0.10500,0.000006,0.1420,0.609,179.984
6,TQG,"KAROL G,Shakira",MAÑANA SERÁ BONITO,4kS7bSuU0Jm9LYMosFU2x5,0DWdj2oZMBFSzRsi2Cvfzf,89,2023-02-24,199440,True,https://open.spotify.com/track/0DWdj2oZMBFSzRs...,0.720,0.630,4,-3.547,0,0.2770,0.67300,0.000000,0.0936,0.607,179.974
7,Punto G,KAROL G,OCEAN,4i5b4YWuMtneUSvQPONwzK,2E82gORVSA7UMEz5M1IP3b,58,2019-05-03,183160,False,https://open.spotify.com/track/2E82gORVSA7UMEz...,0.791,0.746,6,-4.716,1,0.0907,0.00938,0.000117,0.1250,0.905,165.994
8,Créeme,"KAROL G,Maluma",OCEAN,4i5b4YWuMtneUSvQPONwzK,4EKZsrsCKyqr64FBHLc0DU,71,2019-05-03,214293,False,https://open.spotify.com/track/4EKZsrsCKyqr64F...,0.715,0.874,6,-3.576,1,0.0548,0.12600,0.000162,0.1050,0.652,96.995
9,Ahora Me Llama - Remix,"KAROL G,Bad Bunny,Quavo",Unstoppable,12nlJpvrOd7tTOaCxB1UeR,5K5mnE8uQnf9zIEX7ojdoq,51,2017-10-27,251787,False,https://open.spotify.com/track/5K5mnE8uQnf9zIE...,0.668,0.613,0,-7.050,0,0.0421,0.04330,0.000045,0.1390,0.218,135.034


In [30]:
music_df.isnull().sum()

track name          0
artists             0
album name          0
album id            0
track id            0
popularity          0
release date        0
duration (ms)       0
explicit            0
external urls       0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
dtype: int64

In [20]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
data = music_df

In [32]:
def calculate_weighted_popularity(release_date):
    release_date = datetime.strptime(release_date,'%Y-%m-%d')
    
    
    time_span = datetime.now() - release_date
    
    weight = 1 / (time_span.days + 1)
    
    return weight



In [33]:
scaler = MinMaxScaler()
music_features = music_df[['danceability','energy','key',
                           'loudness','mode','speechiness',
                          'acousticness','instrumentalness','liveness',
                           'tempo']].values

music_features_scaled = scaler.fit_transform(music_features)

In [37]:
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['track name'].values:
        print(f"'{input_song_name}' not found in the database. Please enter valid song name")
        return
    
    input_song_index = music_df[music_df['track name'] == input_song_name].index[0]
    
    similarty_scores = cosine_similarity([music_features_scaled[input_song_index]],
                                         music_features_scaled)
    
    similar_song_indices = similarty_scores.argsort()[0][::-1][1:num_recommendations +1]
    
    content_based_recommendations = music_df.iloc[similar_song_indices][['track name','artists','album name','release date','popularity']]
    
    return content_based_recommendations

In [44]:
content_based_recommendations("Me Enamoré")

,track name,artists,album name,release date,popularity
12,EL BAÑO,"Enrique Iglesias,Bad Bunny",FINAL (Vol.1),2021-09-17,58
1,El Préstamo,Maluma,F.A.M.E.,2018-05-18,64
3,Chantaje (feat. Maluma),"Shakira,Maluma",El Dorado,2017-05-26,78
14,Sin Pijama,"Becky G,NATTI NATASHA",Sin Pijama,2018-04-20,68
22,Reggaetón Lento (Bailemos),CNCO,Primera Cita,2016-08-26,74


In [51]:
def hybrid_recommendations(input_song_name, num_recommendations = 5, alpha=0.5):
    if input_song_name not in  music_df['track name'].values:
        print(f"'{input_song_name}' not found in the database. Please enter valid song name")
        return
    
    content_based_rec = content_based_recommendations(input_song_name,num_recommendations)
    
    popularity_score = music_df.loc[music_df['track name'] == input_song_name,'popularity'].values[0]
    
    weighted_popularity_score = popularity_score * calculate_weighted_popularity(music_df.loc[music_df['track name'] == input_song_name,'release date' ].values[0])
    
    hybrid_recommendations = content_based_rec
    hybrid_recommendations = hybrid_recommendations._append({
        'track name':input_song_name,
        'artists':music_df.loc[music_df['track name'] == input_song_name,'artists'].values[0],
        'album name':music_df.loc[music_df['track name'] == input_song_name,'album name'].values[0],
        'release date': music_df.loc[music_df['track name'] == input_song_name,'release date' ].values[0],
        'popularity':weighted_popularity_score
    },ignore_index = True)
    
    hybrid_recommendations = hybrid_recommendations.sort_values(by='popularity',ascending=False)
    
    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['track name'] != input_song_name]
    
    return hybrid_recommendations

In [67]:
hybrid_recommendations("FULANITO",22,0.6)

,track name,artists,album name,release date,popularity
16,TQG,"KAROL G,Shakira",MAÑANA SERÁ BONITO,2023-02-24,89.0
8,Hawái,Maluma,PAPI JUANCHO,2020-08-21,80.0
0,Chantaje (feat. Maluma),"Shakira,Maluma",El Dorado,2017-05-26,78.0
14,El Perdedor,Maluma,"Pretty Boy, Dirty Boy",2015-10-30,77.0
18,Mi Gente,"J Balvin,Willy William",Vibras,2018-05-25,75.0
4,Reggaetón Lento (Bailemos),CNCO,Primera Cita,2016-08-26,74.0
19,BICHOTA,KAROL G,KG0516,2021-03-26,73.0
20,Safari,"J Balvin,Pharrell Williams,BIA,Sky Rompiendo",Energía,2016-06-24,72.0
11,Corazón (feat. Nego do Borel),"Maluma,Nego do Borel",F.A.M.E.,2018-05-18,72.0
9,Say My Name,"David Guetta,Bebe Rexha,J Balvin",7,2018-09-14,71.0
